In [17]:
import requests
from bs4 import BeautifulSoup
import glob, os
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import urllib
from urllib.parse import urlencode, quote_plus
import requests
from bs4 import BeautifulSoup
import json
from pandas import json_normalize

### 공공데이터 포털 API 이용하기
- 인코딩키 : %2FVg1J8MmKzuTqcg1RFWfc3U7B6OLQGu28VZvd9EkkL7l0%2BIYsdB5Oe0eTm85cUMsNyjZVrFo7GC8fKNxLqKRDA%3D%3D
- 디코딩키 : /Vg1J8MmKzuTqcg1RFWfc3U7B6OLQGu28VZvd9EkkL7l0+IYsdB5Oe0eTm85cUMsNyjZVrFo7GC8fKNxLqKRDA==

## RFID 기반 음식물쓰레기 종합배출내역(배출량, 배출횟수)을 배출일별 배출내역 정보 제공
- https://www.data.go.kr/data/3045421/openapi.do
- 배출년	disYear
- 배출월	disMonth	
- 배출일	disDate
- 배출요일	disDay
- 배출량	disQuantity
- 배출량비율 disQuantityRate
- 배출횟수	disCount
- 배출횟수비율	disCountRate

In [26]:
def get_일별종합배출내역(year,month):
    ServiceKey = "/Vg1J8MmKzuTqcg1RFWfc3U7B6OLQGu28VZvd9EkkL7l0+IYsdB5Oe0eTm85cUMsNyjZVrFo7GC8fKNxLqKRDA=="
    open_url = 'http://apis.data.go.kr/B552584/RfidFoodWasteServiceNew/getTotalDateList'
    queryParams = '?' + urlencode({ quote_plus('ServiceKey') : ServiceKey,
                                   quote_plus('type') : 'xml',
                                   quote_plus('disYear') : year,
                                   quote_plus('disMonth') : month,
                                   quote_plus('page') : '1',
                                   quote_plus('rowNum') : '31' })

    open_url_get = open_url + queryParams
    res = requests.get(open_url_get)

    datas = BeautifulSoup(res.content,"html.parser").find_all("list")
    일별종합배출내역 = pd.DataFrame()

    날짜 = []; 요일 = [] ; 배출량 = [] ; 배출량비율 = [] ; 배출횟수 = [] ; 배출횟수비율 = []
    for data in datas:
        날짜.append(data.find("disyear").get_text() + "-" + data.find("dismonth").get_text() + "-" + data.find("disdate").get_text())
        요일.append(data.find("disday").get_text())
        배출량.append(data.find("disquantity").get_text())
        배출량비율.append(data.find("disquantityrate").get_text())
        배출횟수.append(data.find("discount").get_text())
        배출횟수비율.append(data.find("discountrate").get_text())

    일별종합배출내역["날짜"] = 날짜 ;일별종합배출내역["요일"] = 요일 ; 일별종합배출내역["배출량"] = 배출량
    일별종합배출내역["배출량비율"] = 배출량비율 ; 일별종합배출내역["배출횟수"] = 배출횟수
    일별종합배출내역["배출횟수비율"] = 배출횟수비율

    return 일별종합배출내역

In [27]:
years = ["2018","2019","2020","2021"]
months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
res_df = []

for year in years:
    for month in months:
        res_df.append(get_일별종합배출내역(year,month))

2021년 5월까지 제공된다

In [32]:
pd.concat(res_df[:-7],axis=0).to_csv("RFID 방식일별종합배출내역.csv",encoding="utf-8-sig",index=False)

## RFID 기반 음식물쓰레기 지자체별 종합배출내역(배출량, 배출횟수)을 배출일별 배출내역 정보 제공
- http://apis.data.go.kr/B552584/RfidFoodWasteServiceNew/getCityDateList
- 배출년	disYear
- 배출월	disMonth	
- 배출일	disDate
- 배출요일	disDay
- 지자체 영역코드	cityCode
- 지자체 시도명	citySidoName
- 지자체 시군구명	citySggName
- 배출량	disQuantity
- 배출량비율 disQuantityRate
- 배출횟수	disCount
- 배출횟수비율	disCountRate

- 제주 제주시 : W6X
- 제주 서귀포시 : W6Y
- 21년 4 6 7 8 월 데이터 존재  x

In [8]:
ServiceKey = "/Vg1J8MmKzuTqcg1RFWfc3U7B6OLQGu28VZvd9EkkL7l0+IYsdB5Oe0eTm85cUMsNyjZVrFo7GC8fKNxLqKRDA=="
open_url = 'http://apis.data.go.kr/B552584/RfidFoodWasteServiceNew/getCityDateList'

In [9]:
def get_지자체별_일별종합배출내역(year,month,code):
    df_lst = []
    # 제주시와 서귀포시 
    queryParams = '?' + urlencode({ quote_plus('ServiceKey') : ServiceKey,
                                    quote_plus('type') : 'json',
                                    quote_plus('disYear') : year,
                                    quote_plus('disMonth') : month,
                                    quote_plus('cityCode') : code,
                                    quote_plus('page') : '1',
                                    quote_plus('rowNum') : '31' })

    open_url_get = open_url + queryParams
    res = requests.get(open_url_get)

    info = json.loads(res.content)
        
    if json.loads(res.content)["data"]["count"] == 0:
        pass
            #return json.loads(res.content)["data"]
    else:
        df = json_normalize(info['data']["list"])
        df_lst.append(df)
        return pd.concat(df_lst,axis=0)

In [11]:
years = ["2018","2019","2020","2021"]
months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
res_df = []

for code in ["W6X","W6Y"]:
    for year in years:
        for month in months:
            res_df.append(get_지자체별_일별종합배출내역(year,month,code))
지차체별_일별종합배출내역 = pd.concat(res_df,axis=0)

In [14]:
지차체별_일별종합배출내역.columns = ["년","월","일","요일","도시코드","도","시군구","배출량",
                         "배출량비율","배출횟수","배출횟수비율"]
지차체별_일별종합배출내역.reset_index(drop=True)

,년,월,일,요일,도시코드,도,시군구,배출량,배출량비율,배출횟수,배출횟수비율
0,2018,01,01,2,W6X,제주특별자치도,제주시,64831400,3.44,26040,3.32
1,2018,01,02,3,W6X,제주특별자치도,제주시,66771200,3.54,27363,3.49
2,2018,01,03,4,W6X,제주특별자치도,제주시,58467850,3.10,24261,3.09
3,2018,01,04,5,W6X,제주특별자치도,제주시,59877000,3.17,25070,3.20
4,2018,01,05,6,W6X,제주특별자치도,제주시,53416200,2.83,22770,2.90
...,...,...,...,...,...,...,...,...,...,...,...
2429,2021,05,27,5,W6Y,제주특별자치도,서귀포시,29468600,3.39,10454,3.51
2430,2021,05,28,6,W6Y,제주특별자치도,서귀포시,26077650,3.00,9050,3.04
2431,2021,05,29,7,W6Y,제주특별자치도,서귀포시,28788250,3.31,9646,3.24
2432,2021,05,30,1,W6Y,제주특별자치도,서귀포시,29040650,3.34,10518,3.53


In [16]:
지차체별_일별종합배출내역.to_csv("RFID 방식 지자체별_일별종합배출내역.csv",encoding="utf-8-sig",index=False)

## RFID 기반 음식물쓰레기 지자체별 요일별 배출내역

In [19]:
ServiceKey = "/Vg1J8MmKzuTqcg1RFWfc3U7B6OLQGu28VZvd9EkkL7l0+IYsdB5Oe0eTm85cUMsNyjZVrFo7GC8fKNxLqKRDA=="
open_url = 'http://apis.data.go.kr/B552584/RfidFoodWasteServiceNew/getCityDayList'

In [20]:
def get_지자체별_요일별종합배출내역(year,month,code):
    df_lst = []
    # 제주시와 서귀포시 
    queryParams = '?' + urlencode({ quote_plus('ServiceKey') : ServiceKey,
                                    quote_plus('type') : 'json',
                                    quote_plus('disYear') : year,
                                    quote_plus('disMonth') : month,
                                    quote_plus('cityCode') : code,
                                    quote_plus('page') : '1',
                                    quote_plus('rowNum') : '31' })

    open_url_get = open_url + queryParams
    res = requests.get(open_url_get)

    info = json.loads(res.content)
        
    if json.loads(res.content)["data"]["count"] == 0:
        pass
            #return json.loads(res.content)["data"]
    else:
        df = json_normalize(info['data']["list"])
        df_lst.append(df)
        return pd.concat(df_lst,axis=0)

In [24]:
years = ["2018","2019","2020","2021"]
months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
res_df = []

for code in ["W6X","W6Y"]:
    for year in years:
        for month in months:
            res_df.append(get_지자체별_요일별종합배출내역(year,month,code))
제주도_요일별종합배출내역 = pd.concat(res_df,axis=0)

In [27]:
제주도_요일별종합배출내역.columns = ["년","월","요일","요일갯수","도시코드","도","시군구","배출량",
                         "일평균배출량","배출횟수","일평균배출횟수"]
제주도_요일별종합배출내역.to_csv("RFID 방식 지자체별_요일별종합배출내역.csv",encoding="utf-8-sig",index=False)

### 일별 장비 배출량 정보
- http://apis.data.go.kr/B552584/RfidFoodWasteServiceNew/getCityAptEqipDayList
- 배출년	disYear
- 배출월	disMonth
- 배출일	disDay
- 지자체 영역코드 cityCode
- 지자체 시도명 cityName
- 아파트코드 aptCode
- 아파트명 aptName
- 장비일련번호 equipCode
- 장비종류코드 typeCode
- 배출량	disQuantity
- 배출횟수 disCount

In [208]:
ServiceKey = "/Vg1J8MmKzuTqcg1RFWfc3U7B6OLQGu28VZvd9EkkL7l0+IYsdB5Oe0eTm85cUMsNyjZVrFo7GC8fKNxLqKRDA=="
open_url = 'http://apis.data.go.kr/B552584/RfidFoodWasteServiceNew/getCityAptEqipDayList'

In [215]:
queryParams = '?' + urlencode({ quote_plus('ServiceKey') : ServiceKey,
                               quote_plus('type') : 'json',
                               quote_plus('disYear') : '2021',
                               quote_plus('disMonth') : '05',
                               quote_plus('cityCode') : 'W03',
                               quote_plus('aptCode') : 'W03005',
                               quote_plus('equipCode') : 'W03-005-01-1',
                               quote_plus('pageNo') : '1', quote_plus('numOfRows') : '10' })
open_url_get = open_url + queryParams
res = requests.get(open_url_get)
info = json.loads(res.content)

In [216]:
if info["data"]["count"] == 0:
    pass
else:
    df = json_normalize(info['data']["list"])

In [217]:
df

,disYear,disMonth,disDay,cityCode,cityName,aptCode,aptName,equipCode,typeCode,disQuantity,disCount
0,2021,05,14,W03,서울특별시 용산구,W03005,대우월드마크용산(BM),W03-005-01-1,11EQUM,2350,2
1,2021,05,15,W03,서울특별시 용산구,W03005,대우월드마크용산(BM),W03-005-01-1,11EQUM,950,2
2,2021,05,16,W03,서울특별시 용산구,W03005,대우월드마크용산(BM),W03-005-01-1,11EQUM,16150,7
3,2021,05,17,W03,서울특별시 용산구,W03005,대우월드마크용산(BM),W03-005-01-1,11EQUM,1150,1
4,2021,05,18,W03,서울특별시 용산구,W03005,대우월드마크용산(BM),W03-005-01-1,11EQUM,3250,1
5,2021,05,19,W03,서울특별시 용산구,W03005,대우월드마크용산(BM),W03-005-01-1,11EQUM,7150,5
6,2021,05,22,W03,서울특별시 용산구,W03005,대우월드마크용산(BM),W03-005-01-1,11EQUM,8700,5
7,2021,05,23,W03,서울특별시 용산구,W03005,대우월드마크용산(BM),W03-005-01-1,11EQUM,18050,13
8,2021,05,25,W03,서울특별시 용산구,W03005,대우월드마크용산(BM),W03-005-01-1,11EQUM,11700,4
9,2021,05,28,W03,서울특별시 용산구,W03005,대우월드마크용산(BM),W03-005-01-1,11EQUM,12400,4


### RFID종량제를 이용하는 아파트 세대 수 및 거점 좌표 정보 목록 제공
-> 데이터가 10건까지 밖에 안 나옴 사용 x
- http://apis.data.go.kr/B552584/RfidFoodWasteServiceNew/getAptLocInfoList
- 지자체 영역코드	cityCode
- 페이지번호	pageNo
- 한 페이지당 데이터 개수	numOfRows

In [77]:
ServiceKey = "/Vg1J8MmKzuTqcg1RFWfc3U7B6OLQGu28VZvd9EkkL7l0+IYsdB5Oe0eTm85cUMsNyjZVrFo7GC8fKNxLqKRDA=="
open_url = 'http://apis.data.go.kr/B552584/RfidFoodWasteServiceNew/getAptLocInfoList'
queryParams = '?' + urlencode({ quote_plus('ServiceKey') : ServiceKey,
                               quote_plus('type') : 'json',
                               quote_plus('cityCode') : 'W6Y',
                               quote_plus('pageNo') : '1',
                               quote_plus('numOfRows') : '100' })
open_url_get = open_url + queryParams
res = requests.get(open_url_get)
info = json.loads(res.content)

- 아파트가 10개 밖에 조회가 안됨...
- 아파트별로 RFID 배출내역 / 장비 정보를 가져오려고 하는데 지금 아파트코드 API가 제대로 작동 안함

### 아파트 배출내역(배출량, 배출횟수)을 일별 정보 목록
- http://apis.data.go.kr/B552584/RfidFoodWasteServiceNew/getCityAptDateList

In [373]:
ServiceKey = "/Vg1J8MmKzuTqcg1RFWfc3U7B6OLQGu28VZvd9EkkL7l0+IYsdB5Oe0eTm85cUMsNyjZVrFo7GC8fKNxLqKRDA=="
open_url = 'http://apis.data.go.kr/B552584/RfidFoodWasteServiceNew/getCityAptDateList'

queryParams = '?' + urlencode({ quote_plus('ServiceKey') : ServiceKey,
                               quote_plus('type') : 'xml',
                               quote_plus('disYear') : '2019',
                               quote_plus('disMonth') : '06',
                               quote_plus('cityCode') : 'W6X',
                               quote_plus("aptCode") : 'W6X000',
                               quote_plus('page') : '1',
                               quote_plus('rowNum') : '10' })

In [374]:
open_url_get = open_url + queryParams
res = requests.get(open_url_get)
datas = BeautifulSoup(res.content,"html.parser").find_all("list")

In [375]:
datas

[]

### 아파트 세대 정보 
- http://apis.data.go.kr/B552584/RfidFoodWasteServiceNew/getAptLocInfoList
- 이 데이터 사용이 안됨 

In [53]:
open_url = 'http://apis.data.go.kr/B552584/RfidFoodWasteServiceNew/getAptLocInfoList'
queryParams = '?' + urlencode({ quote_plus('ServiceKey') : ServiceKey, quote_plus('type') : 'json',
                               quote_plus('cityCode') : 'W6Y',
                               quote_plus('pageNo') : '2',
                               quote_plus('numOfRows') : '1000' })

In [54]:
open_url_get = open_url + queryParams
res = requests.get(open_url_get)
info = json.loads(res.content)

### 아파트 정보 목록 제공
- http://apis.data.go.kr/B552584/RfidFoodWasteServiceNew/getAptlist

In [59]:
ServiceKey = "/Vg1J8MmKzuTqcg1RFWfc3U7B6OLQGu28VZvd9EkkL7l0+IYsdB5Oe0eTm85cUMsNyjZVrFo7GC8fKNxLqKRDA=="
open_url = 'http://apis.data.go.kr/B552584/RfidFoodWasteServiceNew/getAptlist'

queryParams = '?' + urlencode({ quote_plus('ServiceKey') : ServiceKey,
                               quote_plus('type') : 'json',
                               quote_plus('cityCode') : 'W6X',
                               quote_plus('page') : '1',
                               quote_plus('rowNum') : '5000' })

open_url_get = open_url + queryParams
res = requests.get(open_url_get)
info = json.loads(res.content)

if info["data"]["count"] == 0:
    pass
else:
    df_1 = json_normalize(info['data']["list"])

In [60]:
queryParams = '?' + urlencode({ quote_plus('ServiceKey') : ServiceKey,
                               quote_plus('type') : 'json',
                               quote_plus('cityCode') : 'W6Y',
                               quote_plus('page') : '1',
                               quote_plus('rowNum') : '5000' })

open_url_get = open_url + queryParams
res = requests.get(open_url_get)
info = json.loads(res.content)

if info["data"]["count"] == 0:
    pass
else:
    df_2 = json_normalize(info['data']["list"])

In [65]:
아파트 = pd.concat([df_1,df_2],axis=0).reset_index(drop=True)
아파트.to_csv("제주도 아파트 정보.csv",encoding="utf-8-sig",index=False)

In [68]:
len(아파트[["aptCode","cityCode"]].value_counts())

2732

## 지자체/아파트 배출내역 목록조회 일별

In [96]:
open_url = 'http://apis.data.go.kr/B552584/RfidFoodWasteServiceNew/getCityAptDateList'
queryParams = '?' + urlencode({ quote_plus('ServiceKey') : ServiceKey,
                               quote_plus('type') : 'json', quote_plus('disYear') : '2020',
                               quote_plus('disMonth') : '06', quote_plus('cityCode') : 'W6X',
                               quote_plus('aptCode') : 'W6X08E', quote_plus('page') : '1',
                               quote_plus('rowNum') : '1' })

In [98]:
open_url_get = open_url + queryParams
res = requests.get(open_url_get)
info = json.loads(res.content)
json_normalize(info['data']["list"])

,disYear,disMonth,disDate,disDay,cityCode,citySidoName,citySggName,aptCode,aptName,disQuantity,disQuantityRate,disCount,disCountRate
0,2020,06,01,2,W6X,제주특별자치도,제주시,W6X08E,조천읍 거치대 조천-19,900,0.6,1,1.3


In [103]:
def get_지자체별_아파트별_일별배출내역(year,month,citycode,aptcode):
    df_lst = []
    # 제주시와 서귀포시 
    queryParams = '?' + urlencode({ quote_plus('ServiceKey') : ServiceKey,
                                   quote_plus('type') : 'json', quote_plus('disYear') : year,
                                   quote_plus('disMonth') : month, quote_plus('cityCode') : citycode,
                                   quote_plus('aptCode') : aptcode, quote_plus('page') : '1',
                                   quote_plus('rowNum') : '1' })

    open_url_get = open_url + queryParams
    res = requests.get(open_url_get)
    info = json.loads(res.content)
        
    if info["data"]["count"] == 0:
        pass
            #return json.loads(res.content)["data"]
    else:
        df = json_normalize(info['data']["list"])
        df_lst.append(df)
        return pd.concat(df_lst,axis=0)

- W6X08E : 호출되는 아파트 ( info["data"]["resultMsg"] == "성공" )
- W6X00D : 호출되지 않는 아파트 ( info["data"]["count"] == 0 ) '페이지 파라메터값을 사용 할수 없습니다.'

In [106]:
result_lst

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
   disYear disMonth disDate  disDay cityCode citySidoName citySggName aptCode  \
 0    2018       01      01       2      W6X      제주특별자치도         제주시  W6X062   
 
     aptName  disQuantity  disQuantityRate  disCount  disCountRate  
 0  외도동거치대31        15500             2.79         2          2.53  ,
   disYear 

In [110]:
아파트[["aptCode","cityCode"]]

,aptCode,cityCode
0,W6X000,W6X
1,W6X001,W6X
2,W6X002,W6X
3,W6X003,W6X
4,W6X004,W6X
...,...,...
2727,W6Y745,W6Y
2728,W6Y746,W6Y
2729,W6Y747,W6Y
2730,W6Y748,W6Y


In [125]:
result_lst = []
for idx,row in 아파트[["aptCode","cityCode"]].iterrows(): 
        result_lst.append(get_지자체별_아파트별_일별배출내역("2018","02",row["cityCode"],row["aptCode"]))

In [127]:
data_2018_02 = pd.concat(result_lst,axis=0)

In [151]:
result_lst = []
for idx,row in 아파트[["aptCode","cityCode"]].iterrows(): 
        result_lst.append(get_지자체별_아파트별_일별배출내역("2021","01",row["cityCode"],row["aptCode"]))

ConnectionError: ('Connection aborted.', ConnectionAbortedError(10053, '현재 연결은 사용자의 호스트 시스템의 소프트웨어의 의해 중단되었습니다', None, 10053, None))

In [149]:
data_2021_01 = pd.concat(result_lst,axis=0)

ValueError: All objects passed were None

In [ ]:
years = ["2018","2019","2020","2021"]
months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
result_lst = []

for year in years:
    for month in months:
        for idx,row in 아파트[["aptCode","cityCode"]].iterrows(): 
            result_lst.append(get_지자체별_아파트별_일별배출내역(year,month,row["cityCode"],row["aptCode"]))

In [124]:
data = pd.concat(result_lst,axis=0)
result = data[data["disMonth"] == "01"]
result.to_csv("지차체,아파트별 배출내역 2018_01.csv",encoding="utf-8-sig",index=False)

지차체,아파트별 배출내역 2018_01.csv
- 제주시의 아파트 목록 중에서 조회가 되는 아파트가 있고, 조회가 안되는 곳이 있음

## 지자체/아파트 배출내역 목록조회 요일별

## 일별 장비 배출량 정보 목록 조회
- http://apis.data.go.kr/B552584/RfidFoodWasteServiceNew/getCityAptEqipDayList

In [138]:
ServiceKey = "/Vg1J8MmKzuTqcg1RFWfc3U7B6OLQGu28VZvd9EkkL7l0+IYsdB5Oe0eTm85cUMsNyjZVrFo7GC8fKNxLqKRDA=="
open_url = 'http://apis.data.go.kr/B552584/RfidFoodWasteServiceNew/getCityAptEqipDayList'

queryParams = '?' + urlencode({ quote_plus('ServiceKey') : ServiceKey,
                               quote_plus('type') : 'json',
                               quote_plus('disYear') : '2020',
                               quote_plus('disMonth') : '02',
                               quote_plus('cityCode') : 'W6Y',
                               quote_plus('aptCode') : 'W6Y543',
                               quote_plus('pageNo') : '1',
                               quote_plus('numOfRows') : '100' })

In [139]:
open_url_get = open_url + queryParams
res = requests.get(open_url_get)
info = json.loads(res.content)
json_normalize(info['data']["list"])

,disYear,disMonth,disDay,cityCode,cityName,aptCode,aptName,equipCode,typeCode,disQuantity,disCount
0,2020,02,01,W6Y,제주특별자치도 서귀포시,W6Y543,나무이야기하우스(민박) 인근,W6Y-543-01-2,11EQUM,14650,2
1,2020,02,02,W6Y,제주특별자치도 서귀포시,W6Y543,나무이야기하우스(민박) 인근,W6Y-543-01-1,11EQUM,2850,2
2,2020,02,02,W6Y,제주특별자치도 서귀포시,W6Y543,나무이야기하우스(민박) 인근,W6Y-543-01-2,11EQUM,17250,2
3,2020,02,03,W6Y,제주특별자치도 서귀포시,W6Y543,나무이야기하우스(민박) 인근,W6Y-543-01-1,11EQUM,4350,2
4,2020,02,03,W6Y,제주특별자치도 서귀포시,W6Y543,나무이야기하우스(민박) 인근,W6Y-543-01-2,11EQUM,10150,2
5,2020,02,04,W6Y,제주특별자치도 서귀포시,W6Y543,나무이야기하우스(민박) 인근,W6Y-543-01-1,11EQUM,1300,1
6,2020,02,04,W6Y,제주특별자치도 서귀포시,W6Y543,나무이야기하우스(민박) 인근,W6Y-543-01-2,11EQUM,6400,1
7,2020,02,05,W6Y,제주특별자치도 서귀포시,W6Y543,나무이야기하우스(민박) 인근,W6Y-543-01-1,11EQUM,4350,1
8,2020,02,05,W6Y,제주특별자치도 서귀포시,W6Y543,나무이야기하우스(민박) 인근,W6Y-543-01-2,11EQUM,15850,2
9,2020,02,06,W6Y,제주특별자치도 서귀포시,W6Y543,나무이야기하우스(민박) 인근,W6Y-543-01-1,11EQUM,2100,1


## 월별 장비 배출량 정보 목록 조회
- 'http://apis.data.go.kr/B552584/RfidFoodWasteServiceNew/getCityAptEqipMonthList'

In [152]:
ServiceKey = "/Vg1J8MmKzuTqcg1RFWfc3U7B6OLQGu28VZvd9EkkL7l0+IYsdB5Oe0eTm85cUMsNyjZVrFo7GC8fKNxLqKRDA=="
open_url = 'http://apis.data.go.kr/B552584/RfidFoodWasteServiceNew/getCityAptEqipMonthList'

queryParams = '?' + urlencode({ quote_plus('ServiceKey') : ServiceKey,
                               quote_plus('type') : 'json',
                               quote_plus('disYear') : '2021',
                               quote_plus('disMonth') : '07',
                               quote_plus('cityCode') : 'W6Y',
                               quote_plus('aptCode') : 'W6Y543',
                               quote_plus('pageNo') : '1',
                               quote_plus('numOfRows') : '10' })

In [153]:
open_url_get = open_url + queryParams
res = requests.get(open_url_get)
info = json.loads(res.content)
json_normalize(info['data']["list"])

,disYear,disMonth,cityCode,cityName,aptCode,aptName,equipCode,typeCode,disQuantity,disCount
0,2021,01,W6Y,제주특별자치도 서귀포시,W6Y543,나무이야기하우스(민박) 인근,W6Y-543-01-2,11EQUM,314250,28
1,2021,01,W6Y,제주특별자치도 서귀포시,W6Y543,나무이야기하우스(민박) 인근,W6Y-543-01-1,11EQUM,376250,34
2,2021,02,W6Y,제주특별자치도 서귀포시,W6Y543,나무이야기하우스(민박) 인근,W6Y-543-01-2,11EQUM,282100,23
3,2021,02,W6Y,제주특별자치도 서귀포시,W6Y543,나무이야기하우스(민박) 인근,W6Y-543-01-1,11EQUM,247050,27
4,2021,03,W6Y,제주특별자치도 서귀포시,W6Y543,나무이야기하우스(민박) 인근,W6Y-543-01-1,11EQUM,515450,50
5,2021,05,W6Y,제주특별자치도 서귀포시,W6Y543,나무이야기하우스(민박) 인근,W6Y-543-01-1,11EQUM,338250,46


## 장비 상태 정보 목록 조회
- http://apis.data.go.kr/B552584/RfidFoodWasteServiceNew/getEquipStatusList

In [131]:
ServiceKey = "/Vg1J8MmKzuTqcg1RFWfc3U7B6OLQGu28VZvd9EkkL7l0+IYsdB5Oe0eTm85cUMsNyjZVrFo7GC8fKNxLqKRDA=="
open_url = 'http://apis.data.go.kr/B552584/RfidFoodWasteServiceNew/getEquipStatusList'

queryParams = '?' + urlencode({ quote_plus('ServiceKey') : ServiceKey,
                               quote_plus('type') : 'json',
                               quote_plus('cityCode') : 'W6X',quote_plus('aptCode') : 'W6X214',
                               quote_plus('page') : '1', quote_plus('rowNum') : '10' })

In [132]:
open_url_get = open_url + queryParams
res = requests.get(open_url_get)
info = json.loads(res.content)
json_normalize(info['data']["list"])

,equipCode,equipName,equipMemo,typeCode,typeName,cityCode,citySidoName,citySggName,aptCode,aptName,...,statusCapacity,statusMotor,statusEtc,statusModem,statusFull,fullWeight,fullVolume,lastUpdate,wgs84X,wgs84Y
0,W6X-214-01-1,용담1동클린하우스21번,용담1동클린하우스21번,11EQUM,개별계량장비,W6X,제주특별자치도,제주시,W6X214,용담1동 클린하우스21번,...,None,OK,None,OK,OK,41,41,20210804070859,0.0,0.0
1,W6X-214-01-2,용담1동클린하우스21번,용담1동클린하우스21번,11EQUM,개별계량장비,W6X,제주특별자치도,제주시,W6X214,용담1동 클린하우스21번,...,None,OK,None,OK,OK,0,0,20210804070829,0.0,0.0
